In [1]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
print ('Reading the orders dataset...')
df_orders = pd.read_csv('data/orders.csv',dtype={'order_id':np.uint32,
                                                 'user_id':np.uint32,
                                                 'order_number':np.uint8,
                                                 'order_dow':np.uint8,
                                                 'order_hour_of_day':np.uint8,
                                                'eval_set':'category'})

Reading the orders dataset...


In [3]:
df_orders['days_since_prior_order'].fillna(0,inplace=True)
df_orders['days_since_prior_order'] = df_orders['days_since_prior_order'].astype(np.uint8)

In [4]:
df_orders['order_hour_of_week'] = 24 * df_orders['order_dow'] + df_orders['order_hour_of_day']

In [5]:
df_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_hour_of_week
0,2539329,1,prior,1,2,8,0,56
1,2398795,1,prior,2,3,7,15,79
2,473747,1,prior,3,3,12,21,84
3,2254736,1,prior,4,4,7,29,103
4,431534,1,prior,5,4,15,28,111


In [6]:
eval_set = 'train'

In [7]:
df_users_eval = df_orders.loc[df_orders.eval_set==eval_set,'user_id']

In [8]:
df_users = df_orders.loc[df_orders.eval_set==eval_set]
del df_users['eval_set']


#df_users.columns = ['_'.join(col).strip() for col in df_users.columns.values]

df_users.rename(columns={'order_number':'train_order_number',
                         'order_dow':'train_order_dow',
                         'order_hour_of_day':'train_order_hod',
                         'order_hour_of_week':'train_order_how',
                         'days_since_prior_order':'train_dspo'},inplace=True)

/usr/local/lib/python2.7/dist-packages/pandas/core/frame.py:2834: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [9]:
df_users.head()

,order_id,user_id,train_order_number,train_order_dow,train_order_hod,train_dspo,train_order_how
10,1187899,1,11,4,8,14,104
25,1492625,2,15,1,11,30,35
49,2196797,5,5,0,11,6,11
74,525192,7,21,2,11,6,59
78,880375,8,4,1,14,10,38


# Products prior

In [10]:
df_orders_prior = df_orders.loc[df_orders.eval_set=='prior']
df_orders_prior = df_orders_prior.loc[df_orders_prior.user_id.isin(df_users_eval)]

## Number of days between the order_number and the last order: Days Before Last Order

The first step is to group the days_since_prior_order per user_id. Then we compute the reverse cumulative sum of the list (i.e. 0 days for last order, maximum number of days for the first order of the user). 

In [11]:
def cumsum_days(group):
    days = (np.cumsum(group.values[:0:-1]))
    return np.insert(days,0,0)
    
ser_dblo = df_orders_prior.groupby('user_id')['days_since_prior_order'].apply(cumsum_days)

In [12]:
ser_dblo.head()

user_id
1          [0, 30, 30, 44, 64, 83, 111, 140, 161, 176]
2    [0, 13, 43, 71, 101, 107, 115, 142, 156, 169, ...
5                                      [0, 19, 29, 40]
7    [0, 7, 10, 17, 45, 56, 58, 75, 80, 87, 93, 102...
8                                          [0, 30, 60]
Name: days_since_prior_order, dtype: object

In [13]:
def calc_days(row):
    return ser_dblo.loc[row['user_id']][-row['order_number']]

pickle_name = 'orders_prior_'+eval_set+'.pkl'
Force = False

if Force == True:
    df_orders_prior['dblo'] = df_orders_prior.apply(calc_days,axis=1)
    df_orders_prior.to_pickle(pickle_name)
else :
    df_orders_prior = pd.read_pickle(pickle_name)

In [14]:
df_orders_prior.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_hour_of_week,dblo
0,2539329,1,prior,1,2,8,0,56,176
1,2398795,1,prior,2,3,7,15,79,161
2,473747,1,prior,3,3,12,21,84,140
3,2254736,1,prior,4,4,7,29,103,111
4,431534,1,prior,5,4,15,28,111,83


##  Build product prior dataset

In [15]:
print('Reading the prior products dataset...')
df_products_prior = pd.read_csv('data/order_products__prior.csv',dtype={'order_id':np.uint32,
                                                                        'product_id':np.uint32,
                                                                        'add_to_cart_order':np.uint8,
                                                                        'reordered':np.bool})

Reading the prior products dataset...


In [16]:
df_products_prior = pd.merge(df_products_prior,df_orders_prior[['order_id',
                                                                'user_id',
                                                                'order_number',
                                                                'order_dow',
                                                                'order_hour_of_day',
                                                                'order_hour_of_week',
                                                                'dblo']],on='order_id')

df_products_prior.rename(columns={'order_dow':'order_dow',
                                  'order_hour_of_day':'order_hod',
                                  'order_hour_of_week':'order_how',
                                  'days_since_prior_order':'dspo'},inplace=True)


In [17]:
number_of_orders = len(df_orders.loc[df_orders.user_id.isin(df_users_eval)]) - len(df_users_eval) 
#number_of_orders = len(df_products_prior.groupby('order_id')) ##same result but slower
print('Number of prior orders associated to the train dataset: {}'.format(number_of_orders))

Number of prior orders associated to the train dataset: 2047377


In [18]:
df_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hod,order_how,dblo
0,2,33120,1,True,202279,3,5,9,129,123
1,2,28985,2,True,202279,3,5,9,129,123
2,2,9327,3,False,202279,3,5,9,129,123
3,2,45918,4,True,202279,3,5,9,129,123
4,2,30035,5,False,202279,3,5,9,129,123


In [19]:
df_cat = pd.read_csv('data/products.csv',
                     usecols=['product_id','aisle_id','department_id'],
                     index_col=['product_id'], 
                     dtype={'aisle_id':np.uint8,'department_id':np.uint8})

In [20]:
df_products_prior = df_products_prior.join(df_cat,on='product_id')
del df_cat

In [21]:
df_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hod,order_how,dblo,aisle_id,department_id
0,2,33120,1,True,202279,3,5,9,129,123,86,16
1,2,28985,2,True,202279,3,5,9,129,123,83,4
2,2,9327,3,False,202279,3,5,9,129,123,104,13
3,2,45918,4,True,202279,3,5,9,129,123,19,13
4,2,30035,5,False,202279,3,5,9,129,123,17,13


## Orders by Product id

In [22]:
df_product = df_products_prior.groupby('product_id')[['add_to_cart_order',
                                                      'reordered']].agg({'add_to_cart_order':np.mean,'reordered':[np.sum,'size']})
df_product.columns = ['_'.join(col).strip() for col in df_product.columns.values]
#df_product['reordered_ratio'] = df_product['reordered_sum'] / df_product['reordered_sum'].sum()
#df_product['reordered_ratio'] = df_product['reordered_ratio'].astype(np.float32)
#df_product.columns = df_product.columns.levels[1]

df_product.rename(columns={'add_to_cart_order_mean':'atco_per_prod',
                            'reordered_sum':'reord_per_prod',
                            'reordered_size':'ord_per_prod'},inplace=True)

df_product['reord_per_prod'] = df_product['reord_per_prod'].astype(np.uint32)
df_product['ord_per_prod'] = df_product['ord_per_prod'].astype(np.uint32)
df_product.reset_index(inplace=True)

In [23]:
df_product.head()

,product_id,atco_per_prod,reord_per_prod,ord_per_prod
0,1,5.647255,754,1202
1,2,10.766667,10,60
2,3,6.378238,146,193
3,4,9.788991,104,218
4,5,5.416667,8,12


In [24]:
#df_products_prior = pd.merge(df_products_prior,df_product,on='product_id')
#del df_product


## Orders by day of week 

In [25]:
df_order_dow = df_products_prior.groupby('order_dow')['reordered'].agg(['sum','count'])


df_order_dow.rename(columns={'sum':'reord_per_dow',
                             'count':'ord_per_dow'},inplace=True)

df_order_dow['reord_per_dow'] = df_order_dow['reord_per_dow'].astype(np.uint32)
df_order_dow['ord_per_dow'] = df_order_dow['ord_per_dow'].astype(np.uint32)
df_order_dow.reset_index(inplace=True)

df_order_dow.head()

#df_products_prior = pd.merge(df_products_prior,df_order_dow,on='order_dow')
#del df_order_dow

,order_dow,reord_per_dow,ord_per_dow
0,0,2315064,3958392
1,1,2174460,3601985
2,2,1581559,2681752
3,3,1428935,2438791
4,4,1422222,2406032


## Orders by hour of day 

In [26]:
df_order_hour_of_day = df_products_prior.groupby('order_hod')['reordered'].agg(['sum','count'])

df_order_hour_of_day.rename(columns={'sum':'reord_per_hod',
                                     'count':'ord_per_hod'},inplace=True)

df_order_hour_of_day['reord_per_hod'] = df_order_hour_of_day['reord_per_hod'].astype(np.uint32)
df_order_hour_of_day['ord_per_hod'] = df_order_hour_of_day['ord_per_hod'].astype(np.uint32)
df_order_hour_of_day.reset_index(inplace=True)

df_order_hour_of_day.head()

#df_products_prior = pd.merge(df_products_prior,df_order_hour_of_day,on='order_hod')
#del df_order_hour_of_day


,order_hod,reord_per_hod,ord_per_hod
0,0,80066,141526
1,1,40944,73752
2,2,25481,45161
3,3,18858,33702
4,4,19923,34559


## Orders by hour of week 

In [27]:
df_order_hour_of_week = df_products_prior.groupby('order_how')['reordered'].agg(['sum','count'])

df_order_hour_of_week.rename(columns={'sum':'reord_per_how',
                                     'count':'ord_per_how'},inplace=True)

df_order_hour_of_week['reord_per_how'] = df_order_hour_of_week['reord_per_how'].astype(np.uint32)
df_order_hour_of_week['ord_per_how'] = df_order_hour_of_week['ord_per_how'].astype(np.uint32)
df_order_hour_of_week.reset_index(inplace=True)

df_order_hour_of_week.head()

#df_products_prior = pd.merge(df_products_prior,df_order_hour_of_week,on='order_how')
#del df_order_hour_of_week

,order_how,reord_per_how,ord_per_how
0,0,14791,25455
1,1,8179,14658
2,2,4842,8653
3,3,3613,6228
4,4,3127,5671


## Orders by aisle id

In [28]:
df_order_aisle = df_products_prior.groupby('aisle_id')['reordered'].agg(['sum','count'])

df_order_aisle.rename(columns={'sum':'reord_per_aisle',
                                     'count':'ord_per_aisle'},inplace=True)

df_order_aisle['reord_per_aisle'] = df_order_aisle['reord_per_aisle'].astype(np.uint32)
df_order_aisle['ord_per_aisle'] = df_order_aisle['ord_per_aisle'].astype(np.uint32)
df_order_aisle.reset_index(inplace=True)

df_order_aisle.head()

,aisle_id,reord_per_aisle,ord_per_aisle
0,1,26823,45333
1,2,25992,52765
2,3,171631,287781
3,4,62353,127342
4,5,11185,39900


## Orders by department id

In [29]:
df_order_department = df_products_prior.groupby('department_id')['reordered'].agg(['sum','count'])

df_order_department.rename(columns={'sum':'reord_per_depart',
                                     'count':'ord_per_depart'},inplace=True)

df_order_department['reord_per_depart'] = df_order_department['reord_per_depart'].astype(np.uint32)
df_order_department['ord_per_depart'] = df_order_department['ord_per_depart'].astype(np.uint32)
df_order_department.reset_index(inplace=True)

df_order_department.head()

,department_id,reord_per_depart,ord_per_depart
0,1,771688,1424216
1,2,9110,22684
2,3,468812,747555
3,4,3922714,6038459
4,5,51926,92605


## Orders by user ID

In [30]:
df_order_user = df_products_prior.groupby('user_id')['reordered'].agg(['sum','count'])


df_order_user.rename(columns={'sum':'reord_per_user',
                              'count':'ord_per_user'},inplace=True)

df_order_user['reord_per_user'] = df_order_user['reord_per_user'].astype(np.uint32)
df_order_user['ord_per_user'] = df_order_user['ord_per_user'].astype(np.uint32)
df_order_user.reset_index(inplace=True)

df_order_user.head()

,user_id,reord_per_user,ord_per_user
0,1,41,59
1,2,93,195
2,5,14,37
3,7,138,206
4,8,13,49


In [31]:
#df_users.to_csv('data/order_users.csv')

## Orders by users and by products id

In [32]:
def mean_days_between_orders(group):
    if(len(group)>1):
        return np.mean(np.diff(list(group.sort_values())))
    else :
        return group.values[0]

Force = False
pickle_name = 'mean_days_between_orders_'+eval_set+'.pkl'

if Force == True:
    mean_days_between_orders = df_products_prior.groupby(['user_id','product_id'])['dblo'].apply(mean_days_between_orders)
    mean_days_between_orders.to_pickle('mean_days_between_orders_train.pkl')
else :
    mean_days_between_orders = pd.read_pickle(pickle_name)
    

In [33]:
mean_days_between_orders.head()

user_id  product_id
1        196           19.555556
         10258         20.125000
         10326         83.000000
         12427         19.555556
         13032         80.500000
Name: dblo, dtype: float64

In [34]:
df_userXproduct = df_products_prior.groupby(['user_id','product_id']).agg({'add_to_cart_order':np.mean,
                                                                'reordered':'sum',
                                                                'order_dow':np.mean,
                                                                'order_hod':np.mean,
                                                                'order_how':np.mean,
                                                                'dblo': np.min,           
                                                                'aisle_id':'last',
                                                                'department_id':'last'})

In [35]:
df_userXproduct.head()

add_to_cart_order  order_hod   order_how  reordered  dblo  \
user_id product_id                                                              
1       196                  1.400000  10.300000   70.300000        9.0     0   
        10258                3.333333  10.555556   71.888889        8.0     0   
        10326                5.000000  15.000000  111.000000        0.0    83   
        12427                3.300000  10.300000   70.300000        9.0     0   
        13032                6.333333   8.000000   72.000000        2.0     0   

                    order_dow  aisle_id  department_id  
user_id product_id                                      
1       196          2.500000        77              7  
        10258        2.555556       117             19  
        10326        4.000000        24              4  
        12427        2.500000        23             19  
        13032        2.666667       121             14

In [36]:
#df_userXproduct.columns = ['_'.join(col).strip() for col in df_userXproduct.columns.values]
#df_userXproduct['reordered_ratio'] = df_userXproduct['reordered_sum'] / df_userXproduct['reordered_count']  
#df_userXproduct['reordered_ratio'] = df_userXproduct['reordered_ratio'].astype(np.float32)

#df_userXproduct.rename(columns={'aisle_id_last':'aisle_id','department_id_last':'department_id'},inplace=True)

In [37]:
df_userXproduct.rename(columns={'add_to_cart_order':'mean_atco',
                               'order_how':'mean_order_how',
                               'reordered':'number_reord',
                               'order_hod':'mean_order_hod',
                               'order_dow':'mean_order_dow',
                               'dblo':'dslo',
                               'aisle_id':'aisle_id',
                               'department_id':'department_id'},inplace=True)
    

    
df_userXproduct['number_reord'] = df_userXproduct['number_reord'].astype(np.uint8)
df_userXproduct['mean_atco'] = df_userXproduct['mean_atco'].astype(np.float32)
df_userXproduct['mean_order_how'] = df_userXproduct['mean_order_how'].astype(np.float32)
df_userXproduct['mean_order_hod'] = df_userXproduct['mean_order_hod'].astype(np.float32)
df_userXproduct['mean_order_dow'] = df_userXproduct['mean_order_dow'].astype(np.float32)

In [38]:
df_userXproduct = pd.concat([df_userXproduct,mean_days_between_orders],axis=1)

In [39]:
df_userXproduct = pd.merge(df_userXproduct.reset_index(),df_order_aisle,on='aisle_id')
df_userXproduct = pd.merge(df_userXproduct,df_order_department,on='department_id').set_index(['user_id','product_id'])

#del df_userXproduct['aisle_id']
#del df_userXproduct['department_id']
#del df_order_aisle
#del df_order_department

In [40]:
#df_userXproduct = df_userXproduct.join(df_users)
#del df_users
df_userXproduct = pd.merge(df_userXproduct.reset_index(),df_users,
                                        on='user_id').set_index(['user_id','product_id'])

In [41]:
df_userXproduct = pd.merge(df_userXproduct.reset_index(),df_order_dow,left_on='train_order_dow',right_on='order_dow')
df_userXproduct = pd.merge(df_userXproduct,df_order_hour_of_day,left_on='train_order_hod',right_on='order_hod')
df_userXproduct = pd.merge(df_userXproduct,df_order_hour_of_week,left_on='train_order_how',right_on='order_how').set_index(['user_id','product_id'])

In [42]:
df_userXproduct.drop(['aisle_id',
                      'department_id',
                     'train_order_dow',
                     'train_order_hod',
                     'train_order_how',
                     'order_id'],axis='columns',inplace=True)

In [43]:
df_userXproduct.to_csv('data/userXproduct.csv')

In [44]:
#del df_userXproduct

In [45]:
df_userXproduct.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8474661 entries, (1, 196) to (72713, 2332)
Data columns (total 22 columns):
mean_atco             float32
mean_order_hod        float32
mean_order_how        float32
number_reord          uint8
dslo                  int64
mean_order_dow        float32
dblo                  float64
reord_per_aisle       uint32
ord_per_aisle         uint32
reord_per_depart      uint32
ord_per_depart        uint32
train_order_number    uint8
train_dspo            uint8
order_dow             int64
reord_per_dow         uint32
ord_per_dow           uint32
order_hod             int64
reord_per_hod         uint32
ord_per_hod           uint32
order_how             int64
reord_per_how         uint32
ord_per_how           uint32
dtypes: float32(4), float64(1), int64(4), uint32(10), uint8(3)
memory usage: 866.2 MB


In [46]:
df_userXproduct.memory_usage()/1000000.

Index                 69.242795
mean_atco             33.898644
mean_order_hod        33.898644
mean_order_how        33.898644
number_reord           8.474661
dslo                  67.797288
mean_order_dow        33.898644
dblo                  67.797288
reord_per_aisle       33.898644
ord_per_aisle         33.898644
reord_per_depart      33.898644
ord_per_depart        33.898644
train_order_number     8.474661
train_dspo             8.474661
order_dow             67.797288
reord_per_dow         33.898644
ord_per_dow           33.898644
order_hod             67.797288
reord_per_hod         33.898644
ord_per_hod           33.898644
order_how             67.797288
reord_per_how         33.898644
ord_per_how           33.898644
dtype: float64

# Train dataset

In [47]:
print('Reading the train products dataset...')
df_products_train = pd.read_csv('data/order_products__train.csv',dtype={'order_id':np.uint32,
                                                                        'product_id':np.uint32,
                                                                        'add_to_cart_order':np.uint8,
                                                                        'reordered':np.bool})

Reading the train products dataset...


In [48]:
del df_products_train['add_to_cart_order']
df_products_train = df_products_train.loc[df_products_train.reordered==True]

In [49]:
df_products_train = df_products_train.merge(df_orders[['order_id','user_id']],on='order_id').sort_values(['user_id','product_id']).set_index(['user_id','product_id'])
del df_products_train['order_id']

In [50]:
df_products_train.head()

reordered
user_id product_id          
1       196             True
        10258           True
        13032           True
        25133           True
        26088           True

In [51]:
df_products_train.to_csv('data/target.csv')